In [111]:
import numpy as np
import pandas as pd
import re

In [112]:
df=pd.read_csv("GynoBK.csv",header=None)
df.columns=["Malpractice","Education","Name","Specialty"]
df.head()

,Malpractice,Education,Name,Specialty
0,None reported,"NY COLLEGE OF OSTEOPATHIC MEDICINE, INSTITUTE ...","ABBOUD, AIMAN",Obstetrics and Gynecology
1,None reported,"AIN SHAMS UNIV, FAC OF MED, CAIRO, EGYPT,\n1995","ABDELMALEK, EHAB",Obstetrics and Gynecology
2,None reported,"TEL AVIV UNIVERSITY, SACKLER SCHOOL OF MEDICIN...","ABULAFIA, OVADIA",Obstetrics and Gynecology/Gynecologic Oncology
3,Malpractice\nLast Updated 03-13-2017\nTO TOP\n...,"SUNY, DWNSTATE M C COLL MED, BROOKLYN NY,\n1988","AHMADI, RAMIN",Obstetrics and Gynecology
4,None reported,"DHAKA MEDICAL COLLEGE, DHAKA UNIVERSITY, BANGL...","AHMED, KAMAL",Obstetrics and Gynecology/Pelvic and Reconstru...


In [113]:
def split_out_year(ed):
    s=re.search(',\n',ed).span()
    school=ed[0:s[0]]
    year= ed[s[1]:]
    return year
def split_out_school(ed):
    s=re.search(',\n',ed).span()
    return ed[0:s[0]]
df['School']=list(map(split_out_school,df.Education))
df['Year']=list(map(split_out_year,df.Education))
df=df.drop("Education",axis=1)
# df.head()

In [114]:
us_states=pd.read_csv("States.csv")
us_states=us_states[us_states.x!='US']
us_states=us_states['x']

In [115]:
df['State'] = df['School'].str.extract('( [A-Z]{2}$)')
df.State=df.State.str.strip()

0

In [116]:
# Finding country where they went to school
ind=-1
countries=[]
for st in df.State:
    ind+=1
    if st in us_states.values:
        countries.append('USA')
    else:
        try:
            country=re.search(', [A-Z]+$',df.School[ind]).group() #{1}[a-z]
            #print(country[2:])
            countries.append(country[2:])#df['School'].str.extract(', [A-Z]{1}[a-z]+$')
        except Exception as e:
            #print(e)
            countries.append('')
            continue
df['Country']=countries
# df[['School','State','Country']]
df

,Malpractice,Name,Specialty,School,Year,State,Country
0,None reported,"ABBOUD, AIMAN",Obstetrics and Gynecology,"NY COLLEGE OF OSTEOPATHIC MEDICINE, INSTITUTE ...",1998,NY,USA
1,None reported,"ABDELMALEK, EHAB",Obstetrics and Gynecology,"AIN SHAMS UNIV, FAC OF MED, CAIRO, EGYPT",1995,NaN,EGYPT
2,None reported,"ABULAFIA, OVADIA",Obstetrics and Gynecology/Gynecologic Oncology,"TEL AVIV UNIVERSITY, SACKLER SCHOOL OF MEDICIN...",1986,NaN,ISRAEL
3,Malpractice\nLast Updated 03-13-2017\nTO TOP\n...,"AHMADI, RAMIN",Obstetrics and Gynecology,"SUNY, DWNSTATE M C COLL MED, BROOKLYN NY",1988,NY,USA
4,None reported,"AHMED, KAMAL",Obstetrics and Gynecology/Pelvic and Reconstru...,"DHAKA MEDICAL COLLEGE, DHAKA UNIVERSITY, BANGL...",1976,NaN,BANGLADESH
5,Malpractice\nLast Updated 02-07-2013\nTO TOP\n...,"AJAH, MARCEL",Obstetrics and Gynecology,"UNIVERSITY OF MO, COLUMBIA SCHOOL OF MEDICINE,...",1993,MO,USA
6,None reported,"AJAYI, OLUSEGUN",Obstetrics and Gynecology,"UNIVERSITY OF IBADAN, COLLEGE OF MEDICINE, IBA...",2001,NaN,NIGERIA
7,None reported,"ALAGHBAND, PEYMAN",Obstetrics and Gynecology,"HAMADAN UNIVERSITY OF MEDICAL SCI, HAMADAN, IRAN",1998,NaN,IRAN
8,Malpractice\nLast Updated 07-28-2016\nTO TOP\n...,"ALDAIA, BASSAM",Obstetrics and Gynecology,"UNIVERSITY OF DAMASCUS, FAC OF MEDICINE, DAMAS...",1972,NaN,SYRIA
9,None reported,"ALQAISI, ILHAM",Obstetrics and Gynecology,"UNIVERSITY OF BAGHDAD, COLLEGE OF MEDICINE, BA...",1972,NaN,IRAQ


In [117]:
## TO DO ### Fix New York - NY
#df[df['Country']==''].School

60         NEW YORK UNIVERSITY MEDICAL COLLEGE, NEW YORK
107                     ST MATTHEW'S UNIV - GRAND CAYMAN
150    COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...
179    COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...
207    COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...
Name: School, dtype: object

In [118]:
def find_num_pay(mal):
    # check Judgments and Arbitration Awards
    start,end =re.search('Number of awards:',mal).span()
    try:
        a= int(mal[end+1:end+2])
    except:
        # check Settlements
        start,end_p =re.search('Number of payments:',mal).span()
        a= int(mal[end_p+1:end_p+2])
    return a

In [119]:
ind=0
num_payments=[]
for m in range(len(df)):
    #print(ind)
    if df.Malpractice[ind]=='None reported':
        num_payments.append(0)
    else:
        a=find_num_pay(df.Malpractice[ind])
        num_payments.append(a)
    ind+=1
df['Num_Payments']=num_payments

In [307]:
test=pd.DataFrame([df.loc[8]],columns=df.columns)
test.date='01-09-1998'
df.loc[8,]

Malpractice     Malpractice\nLast Updated 07-28-2016\nTO TOP\n...
Name                                               ALDAIA, BASSAM
Specialty                               Obstetrics and Gynecology
School          UNIVERSITY OF DAMASCUS, FAC OF MEDICINE, DAMAS...
Year                                                         1972
State                                                         NaN
Country                                                     SYRIA
Num_Payments                                                    1
ind_law_suit                                                    1
date                                                            1
area                                                            1
significance                                                    1
Name: 8, dtype: object

In [316]:
# df_try=df.loc[11]
# df.append([df_try]*5,ignore_index=True)

#create empty columns
df['ind_law_suit']=df['Num_Payments']
df['date']=df['Num_Payments']
df['area']=df['Num_Payments']
df['significance']=df['Num_Payments']

Malpractice     Malpractice\nLast Updated 12-30-2014\nTO TOP\n...
Name                                                ELLIS, EUGENE
Specialty                               Obstetrics and Gynecology
School                      NEW YORK MEDICAL COLLEGE, VALHALLA NY
Year                                                         1977
State                                                          NY
Country                                                       USA
Num_Payments                                                    4
ind_law_suit                                                    4
date                                                            4
area                                                            4
significance                                                    4
Name: 61, dtype: object

In [397]:
sum(df.Num_Payments)

158

In [399]:
# creates empty data frame to fill
mal_rows=pd.DataFrame(columns=df.columns)
for ind in range(len(df)):
    #print(ind)
    mal=df.Malpractice[ind]
    if mal=='None reported':
        continue
    start,end =re.search('High   |   Low\n',mal).span()
    start2,end2=re.search("\n\nThe doctor\'s malpractice history",mal).span()
    deets=mal[end:start2].strip()
    deets=deets.split('\n')
    if re.search('None reported',deets[1]) != None:
        x,y=re.search('\n(..-..-....)',mal).span()
        x2,y2=re.search('\n\nSettlements',mal).span()
        deets=[0,mal[x+1:x2]]
    tmp_mal_rows=extract_law_suit_info(deets,ind)
    mal_rows=pd.concat([mal_rows, tmp_mal_rows], axis=0)
bueno_rows=df[df.Malpractice=="None reported"]
new_df=pd.concat([bueno_rows,mal_rows],axis=0)
new_df

[0, '03-05-2012 Kings County, NY Above Average']


,Malpractice,Name,Specialty,School,Year,State,Country,Num_Payments,ind_law_suit,date,area,significance
0,None reported,"ABBOUD, AIMAN",Obstetrics and Gynecology,"NY COLLEGE OF OSTEOPATHIC MEDICINE, INSTITUTE ...",1998,NY,USA,0,0,0,0,0
1,None reported,"ABDELMALEK, EHAB",Obstetrics and Gynecology,"AIN SHAMS UNIV, FAC OF MED, CAIRO, EGYPT",1995,NaN,EGYPT,0,0,0,0,0
2,None reported,"ABULAFIA, OVADIA",Obstetrics and Gynecology/Gynecologic Oncology,"TEL AVIV UNIVERSITY, SACKLER SCHOOL OF MEDICIN...",1986,NaN,ISRAEL,0,0,0,0,0
4,None reported,"AHMED, KAMAL",Obstetrics and Gynecology/Pelvic and Reconstru...,"DHAKA MEDICAL COLLEGE, DHAKA UNIVERSITY, BANGL...",1976,NaN,BANGLADESH,0,0,0,0,0
6,None reported,"AJAYI, OLUSEGUN",Obstetrics and Gynecology,"UNIVERSITY OF IBADAN, COLLEGE OF MEDICINE, IBA...",2001,NaN,NIGERIA,0,0,0,0,0
7,None reported,"ALAGHBAND, PEYMAN",Obstetrics and Gynecology,"HAMADAN UNIVERSITY OF MEDICAL SCI, HAMADAN, IRAN",1998,NaN,IRAN,0,0,0,0,0
9,None reported,"ALQAISI, ILHAM",Obstetrics and Gynecology,"UNIVERSITY OF BAGHDAD, COLLEGE OF MEDICINE, BA...",1972,NaN,IRAQ,0,0,0,0,0
10,None reported,"ALVAREZ, ELSIE",Obstetrics and Gynecology,"SUNY UPSTATE MEDICAL UNIVERSITY, SYRACUSE NY",1990,NY,USA,0,0,0,0,0
12,None reported,"APOSTOL, RADU",Obstetrics and Gynecology,"NY COLLEGE OF OSTEOPATHIC MEDICINE, INSTITUTE ...",2005,NY,USA,0,0,0,0,0
13,None reported,"APPLEWHITE, LIAT",Obstetrics and Gynecology/Maternal-Fetal Medicine,"STONY BROOK UNIVERSITY HEALTH SCIENCES CENTER,...",1989,NY,USA,0,0,0,0,0


In [393]:
def extract_law_suit_info(deets,ind):
    tmp_mal_rows=pd.DataFrame(columns=df.columns)
    for d in range(1,len(deets)):
        suit_info={}
        date=re.search('(..-..-....)', deets[d])
        suit_info['date'] = date.group()
        # area might be zip code or County, State
        try:
            area=re.search('\d{5}',deets[d].strip())
            suit_info['area'] = area.group()
            suit_info['significance']= deets[d].strip()[area.span()[1]+1:]
        except:
            try:
                _,end=date.span()
                area_obj=re.search('[A-Za-z]* *[\w]*(, NY)',deets[d][end:].strip())
                suit_info['area']=area_obj.group().strip()
                suit_info['significance']=deets[d][end:].strip()[area_obj.span()[1]+1:]
            except:
                _,end=date.span()
                area_obj=re.search('Not Available',deets[d][end:].strip())
                suit_info['area']=area_obj.group().strip()
                suit_info['significance']=deets[d][end:].strip()[area_obj.span()[1]+1:]
        tmp_row=pd.DataFrame([df.loc[ind]],columns=df.columns)
        tmp_row['date']=suit_info['date']
        tmp_row['area']=suit_info['area']
        tmp_row['significance']=suit_info['significance']
        tmp_mal_rows=pd.concat([tmp_mal_rows, tmp_row], axis=0)
    return tmp_mal_rows

In [337]:
# this works now let's loop through everyone
ind=11
mal=df.Malpractice[ind]
mal_rows=pd.DataFrame(columns=df.columns)
start,end =re.search('High   |   Low\n',mal).span()
start2,end2=re.search("\n\nThe doctor\'s malpractice history",mal).span()
deets=mal[end:start2].strip()
deets=deets.split('\n')
for d in range(1,len(deets)):
    suit_info={}
    date=re.search('(..-..-....)', deets[d])
    suit_info['date'] = date.group()
    # area might be zip code or County, State
    try:
        area=re.search('\d{5}',deets[d].strip())
        suit_info['area'] = area.group()
        suit_info['significance']= deets[d].strip()[area.span()[1]+1:]
    except:
        try:
            _,end=date.span()
            area_obj=re.search('[A-Za-z]* *[\w]*(, NY)',deets[d][end:].strip())
            suit_info['area']=area_obj.group().strip()
            suit_info['significance']=deets[d][end:].strip()[area_obj.span()[1]+1:]
        except:
            _,end=date.span()
            area_obj=re.search('Not Available',deets[d][end:].strip())
            suit_info['area']=area_obj.group().strip()
            suit_info['significance']=deets[d][end:].strip()[area_obj.span()[1]+1:]
    # write new row with dictionary
    tmp_row=pd.DataFrame([df.loc[ind]],columns=df.columns)
    tmp_row['date']=suit_info['date']
    tmp_row['area']=suit_info['area']
    tmp_row['significance']=suit_info['significance']
    mal_rows=pd.concat([mal_rows, tmp_row], axis=0)

In [391]:
ind=62#3
mal=df.Malpractice[ind]
start,end =re.search('Significance\nof Payment\n(..-..-....)',mal).span()
start2,end2=re.search("\n\nThe doctor\'s malpractice history",mal).span()
# deets=mal[end:start2].strip()
# deets=deets.split('\n')
# test=deets[2].strip()
# _,end=re.search('(..-..-....)', deets[1]).span()
# test=deets[2][end:].strip()
# deets
# re.search('Not Available',test).group()#(, NY)


# area=re.search('[A-Za-z]* *[\w]*(, NY)',deets[1][end:].strip()).group().strip()
# area_obj=re.search('[A-Za-z]* *[\w]*(, NY)',deets[d][end:].strip())
# start,end=area_obj.span()
# # deets[d][area_obj.span()[1]:].strip()
# deets[d][start:end]
x,y=re.search('\n(..-..-....)',mal).span()
x2,y2=re.search('\n\nSettlements',mal).span()
deets=[mal[x+2:x2]]

IndexError: list index out of range

In [338]:
df.loc[62]

Malpractice     Malpractice\nLast Updated 04-09-2012\nTO TOP\n...
Name                                           ELLISON, THOMASENA
Specialty                               Obstetrics and Gynecology
School                   MOREHOUSE SCHOOL OF MEDICINE, ATLANTA GA
Year                                                         1988
State                                                          GA
Country                                                       USA
Num_Payments                                                    1
ind_law_suit                                                    1
date                                                            1
area                                                            1
significance                                                    1
Name: 62, dtype: object

In [360]:
df.Malpractice[62]

'Malpractice\nLast Updated 04-09-2012\nTO TOP\nThere is important information below to help you understand medical malpractice. This physician may have provided important additional information in the physician statement section of this profile.\nJudgments and Arbitration Awards\nNumber of awards: 1\nPayment Details\nDate of Judgment\nor Award\nZip Code or\nCounty of Event\nSignificance\nof Payment\n03-05-2012 Kings County, NY Above Average\n\nSettlements\nSettlement payments will appear in this profile only if the total number of settlements made within the past ten years is equal to three or more. Settlement of a claim may occur for a variety of reasons, which does not necessarily reflect negatively on the professional competence or conduct of the physician. A payment in settlement of a medical malpractice action or claim does not necessarily mean that a medical malpractice has occurred. When a settlement payment is made, the patient agrees to accept some money and stop the legal act

In [ ]:
# JUST IN CASE
# creates empty data frame to fill
mal_rows=pd.DataFrame(columns=df.columns)
for ind in range(len(df)):
    #print(ind)
    mal=df.Malpractice[ind]
    if mal=='None reported':
        continue
    start,end =re.search('High   |   Low\n',mal).span()
    start2,end2=re.search("\n\nThe doctor\'s malpractice history",mal).span()
    deets=mal[end:start2].strip()
    deets=deets.split('\n')
    for d in range(1,len(deets)):
        suit_info={}
        date=re.search('(..-..-....)', deets[d])
        suit_info['date'] = date.group()
        # area might be zip code or County, State
        try:
            area=re.search('\d{5}',deets[d].strip())
            suit_info['area'] = area.group()
            suit_info['significance']= deets[d].strip()[area.span()[1]+1:]
        except:
            try:
                _,end=date.span()
                area_obj=re.search('[A-Za-z]* *[\w]*(, NY)',deets[d][end:].strip())
                suit_info['area']=area_obj.group().strip()
                suit_info['significance']=deets[d][end:].strip()[area_obj.span()[1]+1:]
            except:
                _,end=date.span()
                area_obj=re.search('Not Available',deets[d][end:].strip())
                suit_info['area']=area_obj.group().strip()
                suit_info['significance']=deets[d][end:].strip()[area_obj.span()[1]+1:]
        # write new row with dictionary
        tmp_row=pd.DataFrame([df.loc[ind]],columns=df.columns)
        tmp_row['date']=suit_info['date']
        tmp_row['area']=suit_info['area']
        tmp_row['significance']=suit_info['significance']
        mal_rows=pd.concat([mal_rows, tmp_row], axis=0)